In [4]:
import os

import pandas as pd

pd.options.display.max_colwidth = 100

/home/ubuntu/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [5]:
prefix = 's3://olgabot-maca/facs/sourmash_index_all/'
txt = 'sourmash_databases.txt'

! aws s3 ls $prefix > $txt
! cat $txt

                           PRE tabula-muris-k21-dna/
                           PRE tabula-muris-k21-protein/
                           PRE tabula-muris-k27-dna/
                           PRE tabula-muris-k27-protein/
                           PRE tabula-muris-k33-dna/
                           PRE tabula-muris-k33-protein/
                           PRE tabula-muris-k51-dna/
                           PRE tabula-muris-k51-protein/


In [26]:
cell = 'A1-B000610-3_56_F-1-1'

signature = 's3://olgabot-maca/facs/sourmash_compute_all/{cell}.sig'

In [17]:
databases = pd.read_table(txt, delim_whitespace=True, header=None, names=['is_prefix', 'database_name'])
databases['database_name'] = databases['database_name'].str.strip('/')
databases = databases.drop('is_prefix', axis=1)
databases['ksize'] = databases['database_name'].str.extract('k(\d+)').astype(int)
databases['sequence_to_compare'] = databases['database_name'].map(lambda x: x.split('-')[-1])
databases['database'] = databases['database_name'].map(lambda x: f'{prefix}/{x}/{x}/')
databases = databases.set_index('database_name')
databases

,ksize,sequence_to_compare,database
database_name,,,
tabula-muris-k21-dna,21,dna,s3://olgabot-maca/facs/sourmash_index_all//tabula-muris-k21-dna/tabula-muris-k21-dna/
tabula-muris-k21-protein,21,protein,s3://olgabot-maca/facs/sourmash_index_all//tabula-muris-k21-protein/tabula-muris-k21-protein/
tabula-muris-k27-dna,27,dna,s3://olgabot-maca/facs/sourmash_index_all//tabula-muris-k27-dna/tabula-muris-k27-dna/
tabula-muris-k27-protein,27,protein,s3://olgabot-maca/facs/sourmash_index_all//tabula-muris-k27-protein/tabula-muris-k27-protein/
tabula-muris-k33-dna,33,dna,s3://olgabot-maca/facs/sourmash_index_all//tabula-muris-k33-dna/tabula-muris-k33-dna/
tabula-muris-k33-protein,33,protein,s3://olgabot-maca/facs/sourmash_index_all//tabula-muris-k33-protein/tabula-muris-k33-protein/
tabula-muris-k51-dna,51,dna,s3://olgabot-maca/facs/sourmash_index_all//tabula-muris-k51-dna/tabula-muris-k51-dna/
tabula-muris-k51-protein,51,protein,s3://olgabot-maca/facs/sourmash_index_all//tabula-muris-k51-protein/tabula-muris-k51-protein/


In [14]:
ignore_abundances = True, False

In [33]:
import itertools

data = list(itertools.product(databases.index, ignore_abundances))

samples = pd.DataFrame(data, columns=['database_name', 'ignore_abundance'])
samples = samples.join(databases, on='database_name')
samples['signature'] = signature
print(samples.shape)
samples.head()

output_prefix = 's3://olgabot-maca/facs/sourmash_search/'

samples['id'] = samples.apply(
    lambda x: '{cell}_{database_name}_ignore-abundance={ignore_abundance}'.format(cell=cell, **x), axis=1)
samples['output'] = output_prefix + samples['id'] + '.csv'
samples = samples.set_index('id')
samples.head()

(16, 6)


,database_name,ignore_abundance,ksize,sequence_to_compare,database,signature,output
id,,,,,,,
A1-B000610-3_56_F-1-1_tabula-muris-k21-dna_ignore-abundance=True,tabula-muris-k21-dna,True,21,dna,s3://olgabot-maca/facs/sourmash_index_all//tabula-muris-k21-dna/tabula-muris-k21-dna/,s3://olgabot-maca/facs/sourmash_compute_all/{cell}.sig,s3://olgabot-maca/facs/sourmash_search/A1-B000610-3_56_F-1-1_tabula-muris-k21-dna_ignore-abundan...
A1-B000610-3_56_F-1-1_tabula-muris-k21-dna_ignore-abundance=False,tabula-muris-k21-dna,False,21,dna,s3://olgabot-maca/facs/sourmash_index_all//tabula-muris-k21-dna/tabula-muris-k21-dna/,s3://olgabot-maca/facs/sourmash_compute_all/{cell}.sig,s3://olgabot-maca/facs/sourmash_search/A1-B000610-3_56_F-1-1_tabula-muris-k21-dna_ignore-abundan...
A1-B000610-3_56_F-1-1_tabula-muris-k21-protein_ignore-abundance=True,tabula-muris-k21-protein,True,21,protein,s3://olgabot-maca/facs/sourmash_index_all//tabula-muris-k21-protein/tabula-muris-k21-protein/,s3://olgabot-maca/facs/sourmash_compute_all/{cell}.sig,s3://olgabot-maca/facs/sourmash_search/A1-B000610-3_56_F-1-1_tabula-muris-k21-protein_ignore-abu...
A1-B000610-3_56_F-1-1_tabula-muris-k21-protein_ignore-abundance=False,tabula-muris-k21-protein,False,21,protein,s3://olgabot-maca/facs/sourmash_index_all//tabula-muris-k21-protein/tabula-muris-k21-protein/,s3://olgabot-maca/facs/sourmash_compute_all/{cell}.sig,s3://olgabot-maca/facs/sourmash_search/A1-B000610-3_56_F-1-1_tabula-muris-k21-protein_ignore-abu...
A1-B000610-3_56_F-1-1_tabula-muris-k27-dna_ignore-abundance=True,tabula-muris-k27-dna,True,27,dna,s3://olgabot-maca/facs/sourmash_index_all//tabula-muris-k27-dna/tabula-muris-k27-dna/,s3://olgabot-maca/facs/sourmash_compute_all/{cell}.sig,s3://olgabot-maca/facs/sourmash_search/A1-B000610-3_56_F-1-1_tabula-muris-k27-dna_ignore-abundan...


Example command:

```
reflow run ~/reflow-workflows/sourmash_compare.rf -signatures=s3://olgabot-maca/facs/sourmash/ -output==s3://olgabot-maca/facs/sourmash_index_k21.npy -ksize=21
```

In [24]:
! sourmash search -h

usage: sourmash [-h] [--traverse-directory] [-q] [--threshold THRESHOLD]
                [--save-matches SAVE_MATCHES] [--best-only] [-n NUM_RESULTS]
                [--containment] [--ignore-abundance] [--scaled SCALED]
                [-o OUTPUT] [-k KSIZE] [--protein] [--no-protein] [--dna]
                [--no-dna]
                query databases [databases ...]

positional arguments:
  query                 query signature
  databases             signatures/SBTs to search

optional arguments:
  -h, --help            show this help message and exit
  --traverse-directory  search all signatures underneath directories.
  -q, --quiet           suppress non-error output
  --threshold THRESHOLD
                        minimum threshold for reporting matches (default=0.08)
  --save-matches SAVE_MATCHES
                        output matching signatures to this file.
  --best-only           report only the best match (with greater speed).
  -n NUM_RESULTS, --num-results NUM_RESULTS
     

## Sourmash search

In [35]:
import json

lines = []

program = 'search'
folder = f'../sourmash/{program}'
! mkdir $folder
            
samples.to_csv(f'{folder}/samples.csv')

config = 	{
		"program": f"../../reflow/sourmash_{program}.rf",
		"runs_file": "samples.csv"
	}


with open(f'{folder}/config.json', 'w') as f:
    json.dump(config, f)
    
! ls -lha $folder

mkdir: cannot create directory ‘../sourmash/search’: File exists
total 20K
drwxrwxr-x  2 ubuntu ubuntu 4.0K Sep 19 17:56 .
drwxrwxr-x 12 ubuntu ubuntu 4.0K Sep 19 17:56 ..
-rw-rw-r--  1 ubuntu ubuntu   74 Sep 19 17:56 config.json
-rw-rw-r--  1 ubuntu ubuntu 5.8K Sep 19 17:56 samples.csv


In [36]:
! head $folder/samples.csv

id,database_name,ignore_abundance,ksize,sequence_to_compare,database,signature,output
A1-B000610-3_56_F-1-1_tabula-muris-k21-dna_ignore-abundance=True,tabula-muris-k21-dna,True,21,dna,s3://olgabot-maca/facs/sourmash_index_all//tabula-muris-k21-dna/tabula-muris-k21-dna/,s3://olgabot-maca/facs/sourmash_compute_all/{cell}.sig,s3://olgabot-maca/facs/sourmash_search/A1-B000610-3_56_F-1-1_tabula-muris-k21-dna_ignore-abundance=True.csv
A1-B000610-3_56_F-1-1_tabula-muris-k21-dna_ignore-abundance=False,tabula-muris-k21-dna,False,21,dna,s3://olgabot-maca/facs/sourmash_index_all//tabula-muris-k21-dna/tabula-muris-k21-dna/,s3://olgabot-maca/facs/sourmash_compute_all/{cell}.sig,s3://olgabot-maca/facs/sourmash_search/A1-B000610-3_56_F-1-1_tabula-muris-k21-dna_ignore-abundance=False.csv
A1-B000610-3_56_F-1-1_tabula-muris-k21-protein_ignore-abundance=True,tabula-muris-k21-protein,True,21,protein,s3://olgabot-maca/facs/sourmash_index_all//tabula-muris-k21-protein/tabula-muris-k21-protein/,s3://olgabot-